In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import basedosdados as bd
import sqlite3 as sqlite

pio.renderers.default='notebook'


Primeiramente, vamos ler nosso banco de dados que contém os dados da geografia, e criar uma tabela onde cada linha contém um ID de município e a sigla do estado ao qual esse município pertence, usando como apoio uma base do pacote "Base dos Dados".

In [ ]:
states = bd.read_sql(
    '''
    SELECT DISTINCT id_municipio AS id, sigla_uf
    FROM  basedosdados.br_bd_diretorios_brasil.setor_censitario AS censo
    GROUP BY id_municipio, sigla_uf
    ''',
    billing_project_id='geobr-analysis')

cities = gpd.read_file("geojson\\brasil.json", encoding='utf-8')

In [ ]:
states.head()

In [ ]:
cities.head()

Com essas tabelas ao nosso dispor, daremos merge dessa tabela de estados com a que contém as informações geográficas de cada município, nos permitindo filtrar os dados de geometria de um estado específico, nesse caso, o do Rio de Janeiro.

In [ ]:
brasil_data = pd.merge(states, cities, on='id')
brasil_data = gpd.GeoDataFrame(brasil_data, geometry='geometry') 

brasil_data.head()

Filtrando apenas para municípios do Rio, temos:

In [ ]:
RJ_data = brasil_data[brasil_data["sigla_uf"] == "RJ"].reset_index(drop=True)
RJ_data = RJ_data.astype({"id": int})

Temos que ler, agora, os dados relacionados à Covid. Esses dados foram retirados do site https://brasil.io/dataset/covid19/caso_full/ .

In [ ]:
covid = pd.read_csv('caso_full.csv.gz', compression='gzip')
covid.rename(columns={"city_ibge_code": "id", "last_available_confirmed_per_100k_inhabitants": "cases_per_100k", "last_available_death_rate": "death_rate"}, inplace=True)
covid = covid[covid["id"] > 1000].astype({"id": int}) # Limpando a base que continha linhas com IDs inexistentes

covid_rj = covid[(covid["state"] == "RJ")].reset_index(drop=True)
covid_rj = covid_rj[covid_rj["date"].str[-2:] == "01"] # Pegando apenas primeiro dia de cada mês
covid_rj = covid_rj[["id", "estimated_population", "cases_per_100k", "death_rate", "date"]] # Pegando apenas colunas relevantes

In [ ]:
covid.head()

Novamente, com as tabelas prontas, basta dar merge e prepará-la para análises:

In [ ]:
RJ_fulldata = pd.merge(RJ_data, covid_rj, on='id')
RJ_fulldata.sort_values(by = "date", inplace=True)
RJ_fulldata["date"] = RJ_fulldata["date"].astype('str')

Vamos, primeiramente, obter uma visão geral do panorama da pandemia 

In [ ]:
fig = px.choropleth(RJ_fulldata,
                   geojson = RJ_fulldata["geometry"],
                   locations = RJ_fulldata.index,
                   color = "death_rate",
                   hover_name = "name",
                   hover_data = ["cases_per_100k", "death_rate", "estimated_population"],
                   labels = {
                       "cases_per_100k": "Cases per 100k habitants",
                       "death_rate": "Mortality rate",
                       "estimated_population": "Estimated population"
                   },
                   color_continuous_scale = "sunsetdark",
                   range_color=(0, 0.25),
                   animation_frame = 'date',
)

fig.update_geos( # Centralizando o mapa no RJ
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.update_layout(
    title_text = "Taxa de mortalidade da Covid-19 ao longo dos meses de pandemia, por município",
    margin={"r":0,"t":10,"l":0,"b":10}
)

fig.show(renderer='notebook_connected')